# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will past the accuracy of your model to the Coursera grader.

This notebook is tested in IBM Watson Studio under python 3.6

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by installing and importing everything we need for this exercise:

In [ ]:
!pip install --upgrade keras-applications keras-preprocessing setuptools tensorflow==1.14.0 keras==2.2.5

In [ ]:
import tensorflow
if not tensorflow.__version__ == '1.14.0':
    print(tensorflow.__version__)
    raise ValueError('please upgrade to TensorFlow 1.14.0, or restart your Kernel (Kernel->Restart & Clear Output)')

import keras
if not keras.__version__ == '2.2.5':
    print(keras.__version__)
    raise ValueError('please upgrade to Keras 2.2.5, or restart your Kernel (Kernel->Restart & Clear Output)')

IMPORTANT! => Please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [6]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [7]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [8]:
model = Sequential()  # Instantiate sequential model
model.add(Dense(512, activation='relu', input_shape=(max_words,))) # Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) # Add second layer
model.add(Dense(num_classes, activation='softmax')) # Add third layer

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [12]:
batch_size = 32

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=5,
    validation_data=(x_test, y_test)
)

score = model.evaluate(x_test, y_test, verbose=0)

W0118 15:01:16.291580 140273311045440 deprecation.py:323] From /opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8982 samples, validate on 2246 samples
Epoch 1/5
8982/8982 [==============================] - 10s 1ms/step - loss: 1.3851 - acc: 0.6926 - val_loss: 0.9622 - val_acc: 0.7823
Epoch 2/5
8982/8982 [==============================] - 10s 1ms/step - loss: 0.7692 - acc: 0.8202 - val_loss: 0.8411 - val_acc: 0.8032
Epoch 3/5
8982/8982 [==============================] - 10s 1ms/step - loss: 0.5450 - acc: 0.8661 - val_loss: 0.8130 - val_acc: 0.7979
Epoch 4/5
8982/8982 [==============================] - 11s 1ms/step - loss: 0.4173 - acc: 0.8963 - val_loss: 0.8186 - val_acc: 0.8068
Epoch 5/5
8982/8982 [==============================] - 11s 1ms/step - loss: 0.3492 - acc: 0.9092 - val_loss: 0.8628 - val_acc: 0.7983


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [13]:
score[1]

0.7983081032947462

Congratulations, now it's time to submit your result to the Coursera grader by executing the following cells (Programming Assingment, Week2). 

We have to install a little library in order to submit to coursera


In [15]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-01-18 15:13:06--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,540       --.-K/s   in 0s      

2020-01-18 15:13:06 (35.7 MB/s) - ‘rklib.py’ saved [2540/2540]



Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [16]:
from rklib import submit
import json

key = "XbAMqtjdEeepUgo7OOVwng"
part = "HCvcp"
email = "belkoviv@student.bmstu.ru"
token = "tlMMfqnlr2vqgOJE" #you can obtain it from the grader page on Coursera (have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)


submit(email, token, 'XbAMqtjdEeepUgo7OOVwng', part, [part], json.dumps(score[1]*100))

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"ozVf2","id":"tE4j0qhMEeecqgpT6QjMdA~ozVf2~CRrMnToFEeqwphK9M2fb0w","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
